In [1]:
import pyspark 
sc = pyspark.SparkContext('local[*]')
# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[756, 912, 644, 54, 658]

In [2]:
from pyspark.sql.functions import sum, col, when, split, concat_ws, broadcast,lit, udf, desc, percent_rank, regexp_replace
from pyspark.sql.types import FloatType
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read CSV Example") \
    .getOrCreate()

In [ ]:
# 5.	Read three datasets by Spark and print counts for each.

In [3]:
# accounts
df_accounts = spark.read \
    .option("header", "true")  \
    .option("inferSchema", "true")  \
    .csv("accounts.csv", sep=";")

df_accounts.count()

500000

In [4]:
# country_abbreviation
df_country_abbreviation = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("country_abbreviation.csv", sep=";") 

df_country_abbreviation.count()

121

In [6]:
# transactions
df_transactions = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("transactions.csv", sep=";")

df_transactions.count()

5000000